In [5]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

* generate igv batch file and snv file pair for each patient, so that you can review all the snv/indels called for a patient. 
* to review, run igv batch command and python script in two separate terminals on gphost08.
* load all bams in igv: (igv -b patient.batch) to load all relevant bam files in igv 
* connect the variants in .snv file with the open igv session: run python command (python /projects/da_workspace/DA-231_SMG_review/variant_igv_review.py -c /projects/tumour_char/analysis_scripts/SVIA/variant_igv_review/tags/v0.2.2/variant_igv_review.cfg -i batch_files/HTMCP-03-06-02063_6.snv -p ./passed_test.txt -f ./failed_test.txt --igv -b ./variants.bed -o)
* record if a variant is real or not and add optional comment, the variants reviewed are separated into two txt file: passed or failed.

# genes to review

In [2]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/immuno_review/'
meta = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf.txt'

# create variant list for review for each patient

In [ ]:
# make a list of variants to review
header = ["#chromosome",  "pos",  "ref",  "alt",  "is_real",  "comments"]

In [6]:
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/mutations_immune_response.txt'
df2 = pd.read_csv(f2, sep='\t')
df2.head(2)

,chr,pos,ref,alt,patient,impact,impact_type,gene
0,6,32548599,C,T,HTMCP-03-06-02001,SYNONYMOUS_CODING,LOW,HLA-DRB1
1,6,29910335,C,T,HTMCP-03-06-02002,NON_SYNONYMOUS_CODING,MODERATE,HLA-A


For the IGV reviewing, they suggested only reviewing the following:

For HLA genes, only consider those that have a HIGH impact type

For non-HLA genes, only consider those that have a HIGH or MODERATE impact type.

In [12]:
df3 = df2[((df2.gene.str.startswith('HLA'))&(df2.impact_type=='HIGH'))|((~df2.gene.str.startswith('HLA'))&(df2.impact_type!='LOW'))]
df3.head(2)
df3.shape

,chr,pos,ref,alt,patient,impact,impact_type,gene
3,3,119256190,G,A,HTMCP-03-06-02058,NON_SYNONYMOUS_CODING,MODERATE,CD80
5,3,121828117,G,C,HTMCP-03-06-02058,NON_SYNONYMOUS_CODING,MODERATE,CD86


(19, 8)

In [26]:
df3.loc[:, 'chromosome'] = df3.loc[:, 'chr']
df3.loc[:, 'is_real'] = 'no'
df3.loc[:, 'comments'] = 'xxx'

In [16]:
patients = df3.patient.tolist()

In [27]:
for p in df3.patient.unique():
    dftmp = df3.query(f'patient=="{p}"')
    n_muts = dftmp.shape[0]
    dftmp.to_csv(f'{wkdir}batch_files/{p}_{n_muts}.snv', sep='\t',index=False)
#     print(p)?

In [14]:
wkdir

'/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/immuno_review/'

# make igv batch files

this is a weird way to do it. but it works. i can simply separate them into tumor and normal and them join two dfs to get the three bams for each patient.

In [53]:
df = pd.read_csv(meta, sep='\t', usecols=['patient', 'DNA_bam', 'RNA_bam'])
df.head(2)

df1 = df.groupby('patient')[['DNA_bam', 'RNA_bam']].apply(lambda x: pd.Series(x.values.ravel().astype(str)[:3]))
df1.columns = ['t_dna', 't_rna', 'n_dna']
df1.head(2)



# write into batch files
for ix, row in df1.iterrows():
    patient = ix
    if patient in patients:
        f = f'{wkdir}batch_files/{patient}.batch'
        with open (f, 'w') as of:
            of.write('new\ngenome hg19\n')
            of.write(' '.join(['load', row['t_dna']]))
            of.write('\n')
            of.write(' '.join(['load', row['t_rna']]))
            of.write('\n')
            of.write(' '.join(['load', row['n_dna']]))

    

,patient,DNA_bam,RNA_bam
0,HTMCP-03-06-02001,/projects/analysis/analysis22/A37234/merge_bwa...,/projects/analysis/analysis22/IX3433/C67GDANXX...
1,HTMCP-03-06-02002,/projects/analysis/analysis22/A37235/merge_bwa...,/projects/analysis/analysis22/IX3432/C67GDANXX...


,t_dna,t_rna,n_dna
patient,,,
HTMCP-03-06-02001,/projects/analysis/analysis22/A37234/merge_bwa...,/projects/analysis/analysis22/IX3433/C67GDANXX...,/projects/analysis/analysis22/A37159/merge_bwa...
HTMCP-03-06-02002,/projects/analysis/analysis22/A37235/merge_bwa...,/projects/analysis/analysis22/IX3432/C67GDANXX...,/projects/analysis/analysis22/A37160/merge_bwa...


16

105

1

...

113

1

153

1

120

# start reviewing process

run the following command in terminal
# start igv session with batch file on gphost08
ssh -X gphost08

igv -b /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/batch_files/HTMCP-03-06-02063.batch
# connect variant list to igv session on gphost08 for review
cd /projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv

python /projects/da_workspace/DA-231_SMG_review/variant_igv_review.py -c /projects/tumour_char/analysis_scripts/SVIA/variant_igv_review/tags/v0.2.2/variant_igv_review.cfg -i batch_files/HTMCP-03-06-02063_6.snv -p ./passed_test.txt -f ./failed_test.txt --igv -b ./variants.bed -o